In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
tf.__version__

'2.4.1'

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):  
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 09m 06s]
val_accuracy: 0.9071666598320007

Best val_accuracy So Far: 0.9096666574478149
Total elapsed time: 00h 43m 52s
INFO:tensorflow:Oracle triggered exit


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        34608     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2973824   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,011,802
Trainable params: 3,011,802
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 185s 109ms/step - loss: 0.1536 - accuracy: 0.9418 - val_loss: 0.2536 - val_accuracy: 0.9130
Epoch 5/10
1688/1688 [==============================] - 216s 128ms/step - loss: 0.1088 - accuracy: 0.9594 - val_loss: 0.2855 - val_accuracy: 0.9080
Epoch 6/10
1688/1688 [==============================] - 199s 118ms/step - loss: 0.0813 - accuracy: 0.9695 - val_loss: 0.3127 - val_accuracy: 0.9068
Epoch 7/10
1688/1688 [==============================] - 228s 135ms/step - loss: 0.0611 - accuracy: 0.9782 - val_loss: 0.3680 - val_accuracy: 0.9147
Epoch 8/10
1688/1688 [==============================] - 228s 135ms/step - loss: 0.0433 - accuracy: 0.9856 - val_loss: 0.4188 - val_accuracy: 0.9127
Epoch 9/10
1688/1688 [==============================] - 227s 134ms/step - loss: 0.0348 - accuracy: 0.9869 - val_loss: 0.4270 - val_accuracy: 0.9137
Epoch 10/10
1688/1688 [==============================] - 205s 121ms/step - loss: 0.0264 - accuracy: 0.9912 - val

In [15]:
model.save("fm_model.h5")